In [3]:
import os
os.environ["TF_NUM_CPUS"] = "4"
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.layers import Dense

In [3]:
import tensorflow as tf
print(tf.config.list_physical_devices('GPU'))


[]


In [34]:
df = pd.read_csv('nifty_dataPEE.csv')

df.set_index('lasttradetime', inplace=True)
scaler = MinMaxScaler(feature_range=(0,1))
df_scaled = scaler.fit_transform(df)
lookback_window = 5
features = df.shape[1]
def create_sequences(data, lookback_window):
  X, y = [], []
  for i in range(len(data) - lookback_window - 1):
    X.append(data[i:(i + lookback_window), :])
    y.append(data[i + lookback_window, :])
  return np.array(X), np.array(y)

X, y = create_sequences(df_scaled, lookback_window)
X_train, y_train = X, y  # Replace with train/validation split if desired
model = Sequential()
model.add(LSTM(units=100, return_sequences=True, input_shape=(lookback_window, features)))
model.add(LSTM(units=100))
model.add(Dense(units=features))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(X_train, y_train, epochs=20,batch_size=30)
number_of_predictions = 50
forecasted_values = []
last_sequence = X_train[-1]

for _ in range(number_of_predictions):
  next_data_point = model.predict(np.expand_dims(last_sequence, axis=0))
  next_data_point_original = scaler.inverse_transform(next_data_point)
  forecasted_values.append(next_data_point_original.flatten())
  last_sequence = np.append(last_sequence[1:], next_data_point, axis=0)

forecast_df = pd.DataFrame(forecasted_values, columns=df.columns)
print(forecast_df['close'].max(),forecast_df['close'].min())


Epoch 1/20


c:\Users\santh\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


72/72 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - loss: 0.0475
Epoch 2/20
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0017
Epoch 3/20
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0017
Epoch 4/20
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0016
Epoch 5/20
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0018
Epoch 6/20
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0012
Epoch 7/20
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0013
Epoch 8/20
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0016
Epoch 9/20
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0015
Epoch 10/20
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0019
Epoch 11/20
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0014
Epoch 12/20
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0013
Epoch 13/20
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0012
Epoch 14/20
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0012
Epoch 15/20
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0012
Epoch 16/20
72/72 ━━━━━━━━━━━━

In [10]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import threading

# Load data
df = pd.read_csv('nifty_dataPEE.csv')

# Preprocessing
df.set_index('lasttradetime', inplace=True)
scaler = MinMaxScaler(feature_range=(0, 1))
df_scaled = scaler.fit_transform(df)
lookback_window = 5
features = df.shape[1]

# Function to create sequences
def create_sequences(data, lookback_window):
    X, y = [], []
    for i in range(len(data) - lookback_window - 1):
        X.append(data[i:(i + lookback_window), :])
        y.append(data[i + lookback_window, :])
    return np.array(X), np.array(y)

# Create sequences
X, y = create_sequences(df_scaled, lookback_window)
X_train, y_train = X, y  # Replace with train/validation split if desired

# Function to train the model and return the trained model
def train_model(X_train, y_train):
    model = Sequential()
    model.add(LSTM(units=100, return_sequences=True, input_shape=(lookback_window, features)))
    model.add(LSTM(units=100))
    model.add(Dense(units=features))
    model.compile(loss='mean_squared_error', optimizer='adam')
    model.fit(X_train, y_train, epochs=5, batch_size=30)
    return model

# Split data for multithreading
num_threads = 4  # Adjust based on your system's capabilities
split_size = len(X_train) // num_threads
thread_list = []
trained_models = []

for i in range(num_threads):
    start_idx = i * split_size
    end_idx = start_idx + split_size if i < num_threads - 1 else len(X_train)
    X_subset = X_train[start_idx:end_idx]
    y_subset = y_train[start_idx:end_idx]
    thread = threading.Thread(target=lambda: trained_models.append(train_model(X_subset, y_subset)))
    thread_list.append(thread)
    thread.start()

for thread in thread_list:
    thread.join()

model = trained_models[-1]

# Continue with prediction code...
# Prediction
number_of_predictions = 50
forecasted_high_values = []
forecasted_low_values = []
last_sequence = X_train[-1]

for _ in range(number_of_predictions):
    next_data_point = model.predict(np.expand_dims(last_sequence, axis=0))
    next_data_point_original = scaler.inverse_transform(next_data_point)
    forecasted_high_values.append(next_data_point_original.flatten()[0])  # Assuming 'high' is the first feature
    forecasted_low_values.append(next_data_point_original.flatten()[1])  # Assuming 'low' is the second feature
    last_sequence = np.append(last_sequence[1:], next_data_point, axis=0)

# Create DataFrame with only 'high' and 'low' forecasts
forecast_df = pd.DataFrame({'high': forecasted_high_values, 'low': forecasted_low_values})

print(forecast_df['high'].max(), forecast_df['low'].min())


c:\Users\santh\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/5
Epoch 1/5
Epoch 1/5
Epoch 1/5
18/18 ━━━━━━━━━━━━━━━━━━━━ 13s 7ms/step - loss: 0.0467
Epoch 2/5
18/18 ━━━━━━━━━━━━━━━━━━━━ 13s 8ms/step - loss: 0.1894
Epoch 2/5
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0041
Epoch 3/5
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0120
Epoch 3/5
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0023
Epoch 4/5
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0049
Epoch 4/5
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0017
Epoch 5/5
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0041
Epoch 5/5
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.00165
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0034
18/18 ━━━━━━━━━━━━━━━━━━━━ 14s 6ms/step - loss: 0.0586
Epoch 2/5
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0040
Epoch 3/5
18/18 ━━━━━━━━━━━━━━━━━━━━ 14s 6ms/step - loss: 0.1368
Epoch 2/5
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0027
Epoch 4/5
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0039
Epoch 3/5
18/18 ━━━━━━━━━

In [36]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense

# Load data
df = pd.read_csv('nifty_dataPEE.csv')

# Set index
df.set_index('lasttradetime', inplace=True)

# Scale data
scaler = MinMaxScaler(feature_range=(0, 1))
df_scaled = scaler.fit_transform(df)

# Parameters
lookback_window = 5
features = df.shape[1]

# Function to create sequences
def create_sequences(data, lookback_window):
    X, y = [], []
    for i in range(len(data) - lookback_window - 1):
        X.append(data[i:(i + lookback_window), :])
        y.append(data[i + lookback_window, :])
    return np.array(X), np.array(y)

# Create sequences
X, y = create_sequences(df_scaled, lookback_window)

# Define training data
X_train, y_train = X, y  # Replace with train/validation split if desired

# Define model
model = Sequential()
model.add(LSTM(units=100, return_sequences=True, input_shape=(lookback_window, features)))
model.add(LSTM(units=100))
model.add(Dense(units=features))
model.compile(loss='mean_squared_error', optimizer='adam')

# Train model
model.fit(X_train, y_train, epochs=20, batch_size=30)

# Number of predictions
number_of_predictions = 50
forecasted_values = []
last_sequence = X_train[-1]

# Generate forecasts
for _ in range(number_of_predictions):
    next_data_point = model.predict(np.expand_dims(last_sequence, axis=0))
    next_data_point_original = scaler.inverse_transform(next_data_point)
    forecasted_values.append(next_data_point_original.flatten())
    last_sequence = np.append(last_sequence[1:], next_data_point, axis=0)

# Create DataFrame for forecasted values
forecast_df = pd.DataFrame(forecasted_values, columns=df.columns)

# Print max and min forecasted values for 'low' and 'high' features
print("Forecasted 'low' - Max:", forecast_df['low'].max(), "Min:", forecast_df['low'].min())
print("Forecasted 'high' - Max:", forecast_df['high'].max(), "Min:", forecast_df['high'].min())


Epoch 1/20


c:\Users\santh\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


72/72 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - loss: 0.0544
Epoch 2/20
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0015
Epoch 3/20
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0022
Epoch 4/20
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0014
Epoch 5/20
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0017
Epoch 6/20
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0015
Epoch 7/20
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0015
Epoch 8/20
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0013
Epoch 9/20
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0013
Epoch 10/20
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0014
Epoch 11/20
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0013
Epoch 12/20
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0012
Epoch 13/20
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0013
Epoch 14/20
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0015
Epoch 15/20
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0012
Epoch 16/20
72/72 ━━━━━━━━━━━━

In [6]:
import numpy as np
import tensorflow as tf

# Set random seed for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

In [32]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf

# Read the CSV file
df = pd.read_csv('nifty_dataPEE.csv')

# Set the index
df.set_index('lasttradetime', inplace=True)

# Scale the data
scaler = MinMaxScaler(feature_range=(0,1))
df_scaled = scaler.fit_transform(df)

lookback_window = 5
features = df.shape[1]

# Function to create sequences
def create_sequences(data, lookback_window):
    X, y = [], []
    for i in range(len(data) - lookback_window - 1):
        X.append(data[i:(i + lookback_window), :])
        y.append(data[i + lookback_window, :])
    return np.array(X), np.array(y)

# Create sequences
X, y = create_sequences(df_scaled, lookback_window)

# Split data for training
X_train, y_train = X, y  # Replace with train/validation split if desired

# Define the model
model = Sequential([
    LSTM(units=100, return_sequences=True, input_shape=(lookback_window, features)),
    LSTM(units=100),
    Dense(units=features)
])

# Compile the model
model.compile(loss='mean_squared_error', optimizer='adam')

# Convert data to TensorFlow Dataset
dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
dataset = dataset.shuffle(buffer_size=len(X_train))  # Shuffle the dataset
dataset = dataset.batch(30)
dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)  # Enable automatic tuning of the number of threads

# Train the model using TensorFlow Dataset
model.fit(dataset, epochs=50)

# Forecasting
number_of_predictions = 50
forecasted_values = []
last_sequence = X_train[-1]

for _ in range(number_of_predictions):
    next_data_point = model.predict(np.expand_dims(last_sequence, axis=0))
    next_data_point_original = scaler.inverse_transform(next_data_point)
    forecasted_values.append(next_data_point_original.flatten())
    last_sequence = np.append(last_sequence[1:], next_data_point, axis=0)

forecast_df = pd.DataFrame(forecasted_values, columns=df.columns)
print(forecast_df['close'].max(), forecast_df['close'].min())


Epoch 1/50


c:\Users\santh\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


72/72 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - loss: 0.0468
Epoch 2/50
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0021
Epoch 3/50
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0017
Epoch 4/50
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0015
Epoch 5/50
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0013
Epoch 6/50
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0015
Epoch 7/50
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0016
Epoch 8/50
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0016
Epoch 9/50
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0012 
Epoch 10/50
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0013
Epoch 11/50
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0015
Epoch 12/50
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0011
Epoch 13/50
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0012
Epoch 14/50
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0013
Epoch 15/50
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0012
Epoch 16/50
72/72 ━━━━━━━━━━━

KeyboardInterrupt: 

In [31]:
import os
os.environ["TF_NUM_INTEROP_THREADS"] = "1"

import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf

# Read the CSV file
df = pd.read_csv('nifty_dataPEE.csv')

# Set the index
df.set_index('lasttradetime', inplace=True)

# Scale the data
scaler = MinMaxScaler(feature_range=(0,1))
df_scaled = scaler.fit_transform(df)

lookback_window = 5
features = df.shape[1]

# Function to create sequences
def create_sequences(data, lookback_window):
    X, y = [], []
    for i in range(len(data) - lookback_window - 1):
        X.append(data[i:(i + lookback_window), :])
        y.append(data[i + lookback_window, :])
    return np.array(X), np.array(y)

# Create sequences
X, y = create_sequences(df_scaled, lookback_window)

# Split data for training
X_train, y_train = X, y  # Replace with train/validation split if desired

# Define the model
model = Sequential([
    LSTM(units=100, return_sequences=True, input_shape=(lookback_window, features)),
    LSTM(units=100),
    Dense(units=features)
])

# Compile the model
model.compile(loss='mean_squared_error', optimizer='adam')

# Convert data to TensorFlow Dataset
dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
dataset = dataset.shuffle(buffer_size=len(X_train))  # Shuffle the dataset
dataset = dataset.batch(30)
dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)  # Enable automatic tuning of the number of threads

# Train the model using TensorFlow Dataset
model.fit(dataset, epochs=50)

# Forecasting
number_of_predictions = 50
forecasted_values = []
last_sequence = X_train[-1]

for _ in range(number_of_predictions):
    next_data_point = model.predict(np.expand_dims(last_sequence, axis=0))
    next_data_point_original = scaler.inverse_transform(next_data_point)
    forecasted_values.append(next_data_point_original.flatten())
    last_sequence = np.append(last_sequence[1:], next_data_point, axis=0)

forecast_df = pd.DataFrame(forecasted_values, columns=df.columns)
print(forecast_df['close'].max(), forecast_df['close'].min())


Epoch 1/50


c:\Users\santh\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


72/72 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - loss: 0.0498
Epoch 2/50
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0016
Epoch 3/50
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0016
Epoch 4/50
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0015
Epoch 5/50
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0015
Epoch 6/50
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0016
Epoch 7/50
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0015
Epoch 8/50
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0016
Epoch 9/50
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0015
Epoch 10/50
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0013
Epoch 11/50
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0014
Epoch 12/50
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0015
Epoch 13/50
24/72 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0011    

KeyboardInterrupt: 

In [ ]:
import tensorflow as tf

# Configure TensorFlow to use multiple CPU threads for data loading
tf.config.threading.set_inter_op_parallelism_threads(4)
tf.config.threading.set_intra_op_parallelism_threads(4)

# Load and preprocess data using tf.data.Dataset with parallelism
train_dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y))
train_dataset = train_dataset.shuffle(buffer_size=1000).batch(batch_size).prefetch(buffer_size=tf.data.AUTOTUNE)

# Define model and compile it
model = tf.keras.Sequential([...])
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model using parallelism
model.fit(train_dataset, epochs=num_epochs)

In [11]:
scaler = MinMaxScaler()
df_scaled = scaler.fit_transform(df)
lookback_window =3
features = 5
def create_sequences(data, lookback_window):
    X, y = [], []
    for i in range(len(data) - lookback_window - 1): 
        X.append(data[i:(i + lookback_window), :])
        y.append(data[i + lookback_window, 0:5]) 
    return np.array(X), np.array(y)
model = Sequential()
model.add(LSTM(units=200, return_sequences=True, input_shape=(lookback_window, features)))
model.add(LSTM(units=200))
model.add(Dense(200, activation=tf.nn.leaky_relu))
model.add(Dense(200, activation=tf.nn.leaky_relu))
model.add(Dense(units=5)) 
model.compile(loss='mean_squared_error', optimizer='adam')
X, y = create_sequences(df_scaled, lookback_window)
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]


model.fit(X_train, y_train, epochs=50, batch_size=30, validation_data=(X_test, y_test))
y_pred = model.predict(X_test)
y_pred_original = scaler.inverse_transform(y_pred)
data = pd.DataFrame(y_pred_original, columns=['open', 'high', 'low' ,'close','volume'])
data 
number_of_predictions = 10
forecasted_values = []
last_sequence = X_train[-1]
for _ in range(number_of_predictions):
  next_data_point = model.predict(np.expand_dims(last_sequence, axis=0))
  next_data_point_original = scaler.inverse_transform(next_data_point)
  forecasted_values.append(next_data_point_original.flatten())
  last_sequence = np.append(last_sequence[1:], next_data_point, axis=0)
forecast_df = pd.DataFrame(forecasted_values, columns=df.columns)
print(forecast_df['close'].max(),forecast_df['close'].min())

c:\Users\santh\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 8s 105ms/step - loss: 0.1644 - val_loss: 0.0545
Epoch 2/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.0307 - val_loss: 0.0458
Epoch 3/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.0236 - val_loss: 0.0194
Epoch 4/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.0144 - val_loss: 0.0117
Epoch 5/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.0116 - val_loss: 0.0129
Epoch 6/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.0095 - val_loss: 0.0074
Epoch 7/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.0082 - val_loss: 0.0086
Epoch 8/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 0.0070 - val_loss: 0.0084
Epoch 9/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 0.0065 - val_loss: 0.0096
Epoch 10/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 0.0064 - val_loss: 0.0083
Epoch 11/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.0063 - val_loss: 0.0082
Epoch 12/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.0062 - val_loss: 0.0080
